In [7]:
#ensemble learning
#정형데이터(db, excel, etc)에 최적화(전처리/특성공학) : ML
#비정형데이터(txt, audio, image) (특성을 찾는 부분, 표현학습) : DL

#여러개의 모델을 합쳐서 하나의 모델을 만들어 냄
#앙상블알고리즘(랜덤포레스트:결정트리를 랜덤하게 해서 숲을 만듬)
#RandomForestClassifier(특성개수의 제곱근만큼의 특성을 선택)/RandomForestRegression(전체특성을 사용)

#분류는 몇 개의 클래스 중 하나로 분류하는 문제곻, 회귀는 임의의 어떤 숫자를 예측하는 문제

#훈련세트 => 랜덤샘플링(중복을허용한샘플링) => 부트스트랩샘플 => 결정트리훈련(모든확률/트리개수, 모든예측/트리개수)

#RandomForest train : 부트스트랩샘플 사용
import pandas as pd
import numpy as np

wine = pd.read_csv('https://bit.ly/wine-date')

wine.head()
wine.info()
wine.describe()

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

rf.fit(train_input, train_target)
print(rf.feature_importances_)

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

#extra tree : : 부트스트랩샘플 사용하지 않음
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(train_input, train_target)
print(et.feature_importances_)

#Gradient boosting : 결정트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동
#randomforest보다 높은 성능을 얻을 수 있다. 하지만, 순서대로 트리를 추가하기 때문에 훈련 속도가 느림
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb.fit(train_input, train_target)
print(gb.feature_importances_)

#histogram-based gradient boosting
#255+1개의 구간으로 나눔
#gradient boosting의 속도와 성능을 개선한 알고리즘
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#permutaion importance
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)
hgb.score(test_input, test_target)

#XGBoost(kaggle) vs LightGBM(microsoft)
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB
0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]
0.8934000384837406
0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]
0.8881086892152563 0.8720430147331015
0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]
0.9321723946453317 0.8801241948619236
[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]
0.8824322471423747 0.8726214185237284
0.9338079582727165 0.8789710890649293
